<a href="https://colab.research.google.com/github/ArkS0001/Improve-RAGs/blob/main/Dense%2BSparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install rank_bm25


In [12]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
import nltk
import numpy as np
from flask import Flask, render_template, request
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import random

nltk.download('punkt')

app = Flask(__name__)

# Load dense embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate 100 random knowledge base sentences
knowledge_base = [
    f"This is a sample sentence about topic {i}." for i in range(1, 101)
]

# Compute dense embeddings
dense_embeddings = embedding_model.encode(knowledge_base)

# Tokenize for sparse embeddings (BM25)
tokenized_docs = [word_tokenize(doc.lower()) for doc in knowledge_base]
bm25 = BM25Okapi(tokenized_docs)

def hybrid_search(query):
    """Performs hybrid search combining dense and sparse retrieval."""
    # Dense retrieval
    query_embedding = embedding_model.encode(query)
    dense_scores = np.dot(dense_embeddings, query_embedding)  # Cosine similarity
    dense_scores = dense_scores / np.max(dense_scores)  # Normalize

    # Sparse retrieval
    tokenized_query = word_tokenize(query.lower())
    sparse_scores = np.array(bm25.get_scores(tokenized_query))
    sparse_scores = sparse_scores / np.max(sparse_scores)  # Normalize

    # Hybrid score (weighted sum)
    hybrid_scores = 0.5 * dense_scores + 0.5 * sparse_scores
    best_match_index = np.argmax(hybrid_scores)
    return knowledge_base[best_match_index], hybrid_scores[best_match_index]

@app.route('/', methods=['GET', 'POST'])
def index():
    result = ""
    if request.method == 'POST':
        query = request.form['query']
        best_match, score = hybrid_search(query)
        result = f"Best Match: {best_match} (Score: {score:.4f})"
    return render_template('index.html', result=result)

if __name__ == '__main__':
    app.run(debug=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [17]:
import nltk
import numpy as np
import random
import ipywidgets as widgets
from IPython.display import display
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Load dense embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate 100 random knowledge base sentences
knowledge_base = [
    f"This is a sample sentence about topic {i}." for i in range(1, 101)
]

# Compute dense embeddings
dense_embeddings = embedding_model.encode(knowledge_base)

# Tokenize for sparse embeddings (BM25)
tokenized_docs = [word_tokenize(doc.lower()) for doc in knowledge_base]
bm25 = BM25Okapi(tokenized_docs)

def hybrid_search(query):
    """Performs hybrid search combining dense and sparse retrieval."""
    # Dense retrieval
    query_embedding = embedding_model.encode(query)
    dense_scores = np.dot(dense_embeddings, query_embedding)  # Cosine similarity
    dense_scores = dense_scores / np.max(dense_scores)  # Normalize

    # Sparse retrieval
    tokenized_query = word_tokenize(query.lower())
    sparse_scores = np.array(bm25.get_scores(tokenized_query))
    max_sparse = np.max(sparse_scores)
    if max_sparse > 0:
        sparse_scores = sparse_scores / max_sparse  # Normalize
    else:
        sparse_scores = np.zeros_like(sparse_scores)  # Avoid NaN

    # Hybrid score (weighted sum)
    hybrid_scores = 0.5 * dense_scores + 0.5 * sparse_scores
    best_match_index = np.argmax(hybrid_scores)
    return knowledge_base[best_match_index], hybrid_scores[best_match_index]

# UI with ipywidgets
def on_button_click(b):
    query = query_input.value
    best_match, score = hybrid_search(query)
    output_label.value = f"Best Match: {best_match} (Score: {score:.4f})"

query_input = widgets.Text(placeholder='Enter your query...')
search_button = widgets.Button(description='Search')
output_label = widgets.Label()
search_button.on_click(on_button_click)

display(query_input, search_button, output_label)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Text(value='', placeholder='Enter your query...')

Button(description='Search', style=ButtonStyle())

Label(value='')

In [18]:
import nltk
import numpy as np
import random
import ipywidgets as widgets
from IPython.display import display
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Load dense embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate a more complex knowledge base with diverse topics
knowledge_base = [
    "Quantum entanglement describes the phenomenon where two particles remain connected, sharing states instantaneously.",
    "Machine learning algorithms can be categorized into supervised, unsupervised, and reinforcement learning models.",
    "The theory of general relativity explains gravity as the curvature of spacetime caused by mass and energy.",
    "Photosynthesis is the process by which plants convert light energy into chemical energy, releasing oxygen.",
    "Blockchain is a decentralized ledger technology that ensures transparency and security in transactions.",
    "The Turing Test evaluates a machine’s ability to exhibit intelligent behavior indistinguishable from a human.",
    "Dark matter is an unknown form of matter that does not emit light but exerts gravitational effects in the universe.",
    "CRISPR technology allows for precise genetic modifications, revolutionizing biotechnology and medicine.",
    "Neural networks, inspired by the human brain, are fundamental to deep learning models used in AI.",
    "Fusion energy, the process powering the Sun, has the potential to provide unlimited clean energy on Earth."
] + [
    f"Topic {i}: This is a more advanced discussion on subject matter {i}, exploring its implications and challenges."
    for i in range(11, 101)
]

# Compute dense embeddings
dense_embeddings = embedding_model.encode(knowledge_base)

# Tokenize for sparse embeddings (BM25)
tokenized_docs = [word_tokenize(doc.lower()) for doc in knowledge_base]
bm25 = BM25Okapi(tokenized_docs)

def hybrid_search(query):
    """Performs hybrid search combining dense and sparse retrieval."""
    # Dense retrieval
    query_embedding = embedding_model.encode(query)
    dense_scores = np.dot(dense_embeddings, query_embedding)  # Cosine similarity
    dense_scores = dense_scores / np.max(dense_scores)  # Normalize

    # Sparse retrieval
    tokenized_query = word_tokenize(query.lower())
    sparse_scores = np.array(bm25.get_scores(tokenized_query))
    max_sparse = np.max(sparse_scores)
    if max_sparse > 0:
        sparse_scores = sparse_scores / max_sparse  # Normalize
    else:
        sparse_scores = np.zeros_like(sparse_scores)  # Avoid NaN

    # Hybrid score (weighted sum)
    hybrid_scores = 0.5 * dense_scores + 0.5 * sparse_scores
    best_match_index = np.argmax(hybrid_scores)
    return knowledge_base[best_match_index], hybrid_scores[best_match_index]

# UI with ipywidgets
def on_button_click(b):
    query = query_input.value
    best_match, score = hybrid_search(query)
    output_label.value = f"Best Match: {best_match} (Score: {score:.4f})"

query_input = widgets.Text(placeholder='Enter your query...')
search_button = widgets.Button(description='Search')
output_label = widgets.Label()
search_button.on_click(on_button_click)

display(query_input, search_button, output_label)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Text(value='', placeholder='Enter your query...')

Button(description='Search', style=ButtonStyle())

Label(value='')

In [19]:
import nltk
import numpy as np
import random
import ipywidgets as widgets
from IPython.display import display
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Load dense embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate an extensive knowledge base with diverse topics
knowledge_base = [
    # Science & Technology
    "Quantum entanglement describes the phenomenon where two particles remain connected, sharing states instantaneously.",
    "The theory of general relativity explains gravity as the curvature of spacetime caused by mass and energy.",
    "Neural networks, inspired by the human brain, are fundamental to deep learning models used in AI.",
    "Blockchain is a decentralized ledger technology that ensures transparency and security in transactions.",
    "CRISPR technology allows for precise genetic modifications, revolutionizing biotechnology and medicine.",
    "The Turing Test evaluates a machine’s ability to exhibit intelligent behavior indistinguishable from a human.",
    "Fusion energy, the process powering the Sun, has the potential to provide unlimited clean energy on Earth.",
    "Dark matter is an unknown form of matter that does not emit light but exerts gravitational effects in the universe.",
    "Photosynthesis is the process by which plants convert light energy into chemical energy, releasing oxygen.",
    "RNA vaccines, like those for COVID-19, use messenger RNA to instruct cells to build an immune response.",
    "Artificial Intelligence is transforming industries through automation, predictive analytics, and data-driven decision-making.",
    "Superconductors allow electricity to flow without resistance at extremely low temperatures.",
    "Graphene is a one-atom-thick layer of carbon that has extraordinary electrical, thermal, and mechanical properties.",
    "Edge computing reduces latency by processing data closer to the source rather than relying on centralized cloud servers.",

    # Space & Astronomy
    "The James Webb Space Telescope is designed to observe the universe’s first galaxies and exoplanets in infrared light.",
    "Black holes have such a strong gravitational pull that nothing, not even light, can escape from them.",
    "Exoplanets are planets outside our solar system, with thousands discovered using the Kepler and TESS telescopes.",
    "Astrobiology is the study of life’s potential in the universe, searching for microbial life on Mars and Europa.",
    "SpaceX successfully launched the Falcon Heavy, a reusable rocket capable of carrying heavy payloads to deep space.",

    # History & Geography
    "The Great Wall of China was built over several centuries to protect against invasions from northern tribes.",
    "The Industrial Revolution marked a period of rapid technological advancements and urbanization in the 18th century.",
    "The Cold War was a period of geopolitical tension between the United States and the Soviet Union from 1947 to 1991.",
    "Ancient Egyptian civilization is known for its pyramids, hieroglyphic writing, and advances in engineering and astronomy.",
    "The Roman Empire was one of history’s most powerful civilizations, known for its roads, architecture, and military strength.",
    "The Silk Road was an ancient trade network connecting China, India, the Middle East, and Europe.",
    "World War II lasted from 1939 to 1945 and involved major global powers, ending with the defeat of Nazi Germany and Japan.",
    "The Renaissance was a cultural movement in Europe that led to advancements in art, science, and literature.",

    # Current Affairs & Economics
    "Cryptocurrencies like Bitcoin and Ethereum use blockchain technology for decentralized transactions.",
    "Inflation occurs when the prices of goods and services rise, reducing purchasing power over time.",
    "Renewable energy sources such as solar, wind, and hydroelectric power are key to reducing carbon emissions.",
    "The global supply chain crisis has led to increased shipping costs and delays due to pandemic-related disruptions.",
    "Electric vehicles (EVs) are becoming more popular as battery technology improves and charging infrastructure expands.",
    "Artificial Intelligence is increasingly being used in finance for fraud detection and automated trading strategies.",
    "The semiconductor chip shortage has affected industries from automotive to consumer electronics worldwide.",
    "E-commerce has experienced rapid growth, with companies like Amazon and Alibaba dominating the market.",

    # Public Knowledge & Daily Life
    "A balanced diet consists of carbohydrates, proteins, fats, vitamins, and minerals for overall health.",
    "Water makes up about 60% of the human body and is essential for survival.",
    "Sleep is crucial for brain function, memory consolidation, and overall physical health.",
    "Exercise helps reduce the risk of chronic diseases like obesity, heart disease, and diabetes.",
    "Vitamin D is essential for bone health and is obtained from sunlight exposure and certain foods.",
    "Recycling helps reduce waste and conserves natural resources by repurposing materials.",
    "The average human heart beats about 100,000 times per day, pumping blood throughout the body.",
    "Plastic pollution is a major environmental issue, affecting marine life and ecosystems.",
    "Social media platforms influence public opinion and have become a key tool for communication and business.",
    "Artificial Intelligence is being used in customer service through chatbots and virtual assistants.",

    # Cybersecurity & Privacy
    "Multi-factor authentication (MFA) enhances security by requiring multiple forms of verification.",
    "Phishing attacks trick users into providing personal information through deceptive emails or websites.",
    "End-to-end encryption ensures that only the sender and recipient can read a message, protecting privacy.",
    "The Tor network allows anonymous browsing by routing traffic through a series of encrypted relays.",
    "Ransomware attacks encrypt victims’ files and demand payment for their release.",
    "Cybersecurity best practices include using strong passwords, updating software, and avoiding suspicious links.",
    "Data breaches can expose sensitive personal information, leading to identity theft and financial fraud.",

    # Medicine & Biology
    "The human genome contains approximately 20,000-25,000 genes that determine physical traits and functions.",
    "Antibiotics help fight bacterial infections but are ineffective against viruses like the common cold.",
    "Stem cells have the potential to develop into different types of cells, offering possibilities for regenerative medicine.",
    "Cancer is a group of diseases characterized by uncontrolled cell growth, which can spread to other body parts.",
    "The brain’s hippocampus is critical for memory formation and learning.",
    "Dopamine is a neurotransmitter that plays a key role in pleasure, motivation, and reward.",
    "CRISPR-Cas9 is a revolutionary gene-editing technology allowing precise modifications of DNA.",
    "Alzheimer’s disease is a neurodegenerative condition that affects memory and cognitive function.",
    "The placebo effect occurs when a patient experiences improvements despite receiving a non-active treatment.",

    # Emerging Technologies
    "5G technology offers higher data speeds and lower latency compared to previous generations of mobile networks.",
    "Quantum computing has the potential to solve complex problems much faster than classical computers.",
    "Autonomous vehicles use sensors and AI to navigate without human intervention.",
    "Nanotechnology involves manipulating materials at the atomic and molecular scale for applications in medicine and industry.",
    "Smart homes use IoT devices to automate lighting, heating, and security systems.",
    "Biometric authentication, such as facial recognition and fingerprint scanning, enhances digital security.",
    "Augmented reality (AR) overlays digital content onto the real world, enhancing user experiences in gaming and retail.",
    "The metaverse is a virtual world where users can interact through digital avatars in immersive environments.",
    "AI-powered drug discovery accelerates the development of new pharmaceuticals by predicting molecule interactions.",

    # Environment & Sustainability
    "Climate change results from human activities like deforestation and burning fossil fuels, leading to global warming.",
    "Carbon capture technology aims to reduce greenhouse gas emissions by storing carbon underground.",
    "Deforestation contributes to biodiversity loss and disrupts ecosystems.",
    "Ocean acidification, caused by increased CO2 levels, harms marine life, including coral reefs.",
    "The Paris Agreement is an international treaty aimed at reducing global carbon emissions to combat climate change.",
    "Biodiversity is essential for ecosystem stability, providing resources like food, medicine, and clean air."
]

# Compute dense embeddings
dense_embeddings = embedding_model.encode(knowledge_base)

# Tokenize for sparse embeddings (BM25)
tokenized_docs = [word_tokenize(doc.lower()) for doc in knowledge_base]
bm25 = BM25Okapi(tokenized_docs)

def hybrid_search(query):
    """Performs hybrid search combining dense and sparse retrieval."""
    # Dense retrieval
    query_embedding = embedding_model.encode(query)
    dense_scores = np.dot(dense_embeddings, query_embedding)  # Cosine similarity
    dense_scores = dense_scores / np.max(dense_scores)  # Normalize

    # Sparse retrieval
    tokenized_query = word_tokenize(query.lower())
    sparse_scores = np.array(bm25.get_scores(tokenized_query))
    max_sparse = np.max(sparse_scores)
    if max_sparse > 0:
        sparse_scores = sparse_scores / max_sparse  # Normalize
    else:
        sparse_scores = np.zeros_like(sparse_scores)  # Avoid NaN

    # Hybrid score (weighted sum)
    hybrid_scores = 0.5 * dense_scores + 0.5 * sparse_scores
    best_match_index = np.argmax(hybrid_scores)
    return knowledge_base[best_match_index], hybrid_scores[best_match_index]

# UI with ipywidgets
def on_button_click(b):
    query = query_input.value
    best_match, score = hybrid_search(query)
    output_label.value = f"Best Match: {best_match} (Score: {score:.4f})"

query_input = widgets.Text(placeholder='Enter your query...')
search_button = widgets.Button(description='Search')
output_label = widgets.Label()
search_button.on_click(on_button_click)

display(query_input, search_button, output_label)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Text(value='', placeholder='Enter your query...')

Button(description='Search', style=ButtonStyle())

Label(value='')